In [ ]:
import cv2
import numpy as np 
from matplotlib import pyplot as plt

input = './data/IMG_2317_maual_cleaned'

In [ ]:
img = cv2.imread(input + ".jpg")
img_thresholded = cv2.inRange(img, (50, 50, 50), (200, 200, 200))

In [ ]:
plt.imshow(img_thresholded, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

In [ ]:
kernel = np.ones((5,5),np.uint8)
opening = cv2.morphologyEx(img_thresholded, cv2.MORPH_OPEN, kernel)

In [ ]:
plt.imshow(opening, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

In [ ]:
image, contours, hierarchy = cv2.findContours(opening.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
print(len(contours))
# print(contours)

In [ ]:
i = 0
for contour in contours:
    (x,y),radius = cv2.minEnclosingCircle(contour)
    center = (int(x),int(y))
    radius = int(radius)
    cv2.circle(img,center,radius,(0,255,0),2)
    # labelling the circles around the centers, in no particular order.
    position = (center[0] - 10, center[1] + 10)
    text_color = (0, 0, 255)
    i += 1
    cv2.putText(img, str(i), position, cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 3)

In [ ]:
plt.imshow(img, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()

In [ ]:
cv2.imwrite(input + "_processed.jpg",img)